In [1]:
import numpy as np
import pandas as pd
import pylab as plt
import h5py
import matplotlib
%matplotlib inline
from astropy.utils.data import download_file  #import file from URL
from matplotlib.ticker import NullFormatter
import astropy.table
import subprocess as sp
import glob
import healpy as hp

In [2]:
flist = glob.glob('/data/des91.b/data/kadrlica/projects/delve/cat/y3t1/v1/cat/cat*.fits')
#generates list of filenames in the directory above

pxnums = [] #initiates pxnums (see below)

for fname in flist:                                         #generates list of ints (pxnums)
    pxnums.append(int(fname.split('.')[1].split('_')[-1])) #containing pixel numbers
    
pxnums = np.array(pxnums) #converts pxnums to array

ra, dec = hp.pix2ang(32, pxnums, lonlat=True) #gets the right ascension and declination
                                              #of each pixel in two arrays

In [3]:
data_boss_north = astropy.table.Table.read('/data/des90.a/data/raulteix/data/speczs/galaxy_DR12v5_CMASSLOWZTOT_North.fits.gz')
#BOSS data for the northern hemisphere

data_boss_south = astropy.table.Table.read('/data/des90.a/data/raulteix/data/speczs/galaxy_DR12v5_CMASSLOWZTOT_South.fits.gz')
#BOSS data for the southern hemisphere

data_boss_all = astropy.table.vstack([data_boss_north, data_boss_south])
#vertical stacking of previous two tables

pxnums_boss = hp.ang2pix(32, data_boss_all['RA'], data_boss_all['DEC'], lonlat=True)
#array of pixel numbers (with repetition for objects in the same pixel!)

overlap_boss_delve = np.in1d(pxnums_boss, pxnums)
#boolean array mask for overlapping pixels in BOSS 


In [4]:
data_boss_good = data_boss_all[overlap_boss_delve]

#all the objects in BOSS that are in pixels also present in DELVE DR2


In [5]:
px_overlap = np.unique(pxnums_boss[overlap_boss_delve])

#all the pixels present in both catalogs

In [6]:
def pixelstyle(pixel):
    if pixel < 10000: return f'0{pixel}' #formats pixel such that they always have 5 digits
    else: return f'{pixel}'

In [7]:
#here we will create a single Astropy Table of the first 800 pixels

dr2_all = []
for pixel in pxnums[6500:7000]: #parallelization
    dr2_i = astropy.table.Table.read(f'/data/des91.b/data/kadrlica/projects/delve/cat/y3t1/v1/cat/cat_hpx_{pixelstyle(pixel)}.fits')
    bandlist = ['G', 'R', 'I', 'Z']
    for band in bandlist:
        dr2_i[f'EXTENDED_CLASS_{band}'] = (1.0*((dr2_i[f'SPREAD_MODEL_{band}'] + 3*dr2_i[f'SPREADERR_MODEL_{band}']) > 0.005) 
        + 1.0*((dr2_i[f'SPREAD_MODEL_{band}'] + dr2_i[f'SPREADERR_MODEL_{band}']) > 0.003) 
        + 1.0*((dr2_i[f'SPREAD_MODEL_{band}'] - dr2_i[f'SPREADERR_MODEL_{band}']) > 0.003))
    dr2_i_f = dr2_i[dr2_i['EXTENDED_CLASS_G']>1] #filtering out likely stars
    dr2_all.append(dr2_i_f)
dr2_all = astropy.table.vstack(dr2_all)

In [8]:
data=dr2_all

In [9]:
dr2_all

QUICK_OBJECT_ID,RA,DEC,HPX2048,NEPOCHS_G,NEPOCHS_R,NEPOCHS_I,NEPOCHS_Z,MAG_PSF_G,MAG_PSF_R,MAG_PSF_I,MAG_PSF_Z,MAGERR_PSF_G,MAGERR_PSF_R,MAGERR_PSF_I,MAGERR_PSF_Z,MAG_AUTO_G,MAG_AUTO_R,MAG_AUTO_I,MAG_AUTO_Z,MAGERR_AUTO_G,MAGERR_AUTO_R,MAGERR_AUTO_I,MAGERR_AUTO_Z,SPREAD_MODEL_G,SPREAD_MODEL_R,SPREAD_MODEL_I,SPREAD_MODEL_Z,SPREADERR_MODEL_G,SPREADERR_MODEL_R,SPREADERR_MODEL_I,SPREADERR_MODEL_Z,CLASS_STAR_G,CLASS_STAR_R,CLASS_STAR_I,CLASS_STAR_Z,FLAGS_G,FLAGS_R,FLAGS_I,FLAGS_Z,WAVG_MAG_PSF_G,WAVG_MAG_PSF_R,WAVG_MAG_PSF_I,WAVG_MAG_PSF_Z,WAVG_MAGERR_PSF_G,WAVG_MAGERR_PSF_R,WAVG_MAGERR_PSF_I,WAVG_MAGERR_PSF_Z,WAVG_MAGRMS_PSF_G,WAVG_MAGRMS_PSF_R,WAVG_MAGRMS_PSF_I,WAVG_MAGRMS_PSF_Z,WAVG_MAG_AUTO_G,WAVG_MAG_AUTO_R,WAVG_MAG_AUTO_I,WAVG_MAG_AUTO_Z,WAVG_MAGERR_AUTO_G,WAVG_MAGERR_AUTO_R,WAVG_MAGERR_AUTO_I,WAVG_MAGERR_AUTO_Z,WAVG_MAGRMS_AUTO_G,WAVG_MAGRMS_AUTO_R,WAVG_MAGRMS_AUTO_I,WAVG_MAGRMS_AUTO_Z,WAVG_SPREAD_MODEL_G,WAVG_SPREAD_MODEL_R,WAVG_SPREAD_MODEL_I,WAVG_SPREAD_MODEL_Z,WAVG_SPREADERR_MODEL_G,WAVG_SPREADERR_MODEL_R,WAVG_SPREADERR_MODEL_I,WAVG_SPREADERR_MODEL_Z,WAVG_SPREADRMS_MODEL_G,WAVG_SPREADRMS_MODEL_R,WAVG_SPREADRMS_MODEL_I,WAVG_SPREADRMS_MODEL_Z,WAVG_FLAGS_G,WAVG_FLAGS_R,WAVG_FLAGS_I,WAVG_FLAGS_Z,EXPNUM_G,EXPNUM_R,EXPNUM_I,EXPNUM_Z,CCDNUM_G,CCDNUM_R,CCDNUM_I,CCDNUM_Z,T_EFF_G,T_EFF_R,T_EFF_I,T_EFF_Z,EXPTIME_G,EXPTIME_R,EXPTIME_I,EXPTIME_Z,A_IMAGE_G,A_IMAGE_R,A_IMAGE_I,A_IMAGE_Z,B_IMAGE_G,B_IMAGE_R,B_IMAGE_I,B_IMAGE_Z,THETA_IMAGE_G,THETA_IMAGE_R,THETA_IMAGE_I,THETA_IMAGE_Z,EBV,EXTINCTION_G,EXTINCTION_R,EXTINCTION_I,EXTINCTION_Z,EXTENDED_CLASS_G,EXTENDED_CLASS_R,EXTENDED_CLASS_I,EXTENDED_CLASS_Z
int64,float64,float64,int64,int16,int16,int16,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,int16,int32,int32,int32,int32,int32,int32,int32,int32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64
10748900000000,3.298231943541636,-12.556033626473209,17595771,2,0,0,0,18.46111,99.0,99.0,99.0,0.0029021911,99.0,99.0,99.0,17.432556,99.0,99.0,99.0,0.002375612,99.0,99.0,99.0,0.0063477373,-1.0,-1.0,-1.0,0.00013443024,1.0,1.0,1.0,0.029664755,-1.0,-1.0,-1.0,3,99,99,99,18.292326,99.0,99.0,99.0,0.0017695251,99.0,99.0,99.0,0.17747532,99.0,99.0,99.0,17.319275,99.0,99.0,99.0,0.0014757162,99.0,99.0,99.0,0.12364064,99.0,99.0,99.0,0.011520251,-1.0,-1.0,-1.0,8.329008e-05,1.0,1.0,1.0,0.0056161555,1.0,1.0,1.0,3,99,99,99,770012,-1,-1,-1,26,-1,-1,-1,0.648166,-1.0,-1.0,-1.0,83.0,-1.0,-1.0,-1.0,7.6646175,-1.0,-1.0,-1.0,4.879676,-1.0,-1.0,-1.0,-58.632526,-1.0,-1.0,-1.0,0.031140175,0.0992126,0.066639975,0.048858937,0.03724365,3.0,1.0,1.0,1.0
10748900000001,3.2953000030678035,-12.57557478781456,17595768,2,1,0,0,23.71661,22.559744,99.0,99.0,0.13478708,0.09432012,99.0,99.0,23.02198,22.057152,99.0,99.0,0.14557356,0.11083797,99.0,99.0,0.013921993,0.01710547,-1.0,-1.0,0.005287032,0.004468336,1.0,1.0,0.48731008,0.54112935,-1.0,-1.0,0,0,99,99,23.326736,22.559742,99.0,99.0,0.08235787,0.09432012,99.0,99.0,0.4117012,0.09432012,99.0,99.0,23.046225,22.057152,99.0,99.0,0.09132877,0.11083797,99.0,99.0,0.026990168,0.11083797,99.0,99.0,0.0083306795,0.01710547,-1.0,-1.0,0.003200379,0.004468336,1.0,1.0,0.00579479,0.004468336,1.0,1.0,2,0,99,99,772292,768708,-1,-1,44,32,-1,-1,0.97666,0.419126,-1.0,-1.0,60.0,59.0,-1.0,-1.0,1.5550553,2.0736136,-1.0,-1.0,0.6394862,0.97934914,-1.0,-1.0,-48.51075,-62.525604,-1

In [10]:
#here we will catalog match the first 800 pixels
#from DELVE DR2 to all of BOSS, then run the matched 
#catalog through BPZ and then callibrate

from astropy.coordinates import SkyCoord
from astropy import units as u
boss_cd = SkyCoord(ra=data_boss_good['RA']*u.degree, dec=data_boss_good['DEC']*u.degree)
delve_cd = SkyCoord(ra=data['RA']*u.deg, dec=data['DEC']*u.deg)
idx, d2d, d3d = boss_cd.match_to_catalog_sky(delve_cd)

In [11]:
len(boss_cd)

944762

In [12]:
good_matches = d2d < 1.0*u.arcsec
print(len(np.unique(idx[good_matches])), np.count_nonzero(good_matches))

0 0


In [13]:
idx_, d2d_, d3d_ = delve_cd.match_to_catalog_sky(boss_cd)

In [14]:
good_matches_ = d2d_ < 1.0*u.arcsec
print(len(np.unique(idx_[good_matches_])), np.count_nonzero(good_matches_))

0 0


In [15]:
data_all = astropy.table.hstack([data_boss_good[good_matches], data[idx][good_matches]])

In [16]:
data_all.write('/data/des90.a/data/raulteix/data/BOSSDELVE/BOSSDELVE14.fits', format='fits')